In [ ]:
%%file BaseClass.py
from copy import deepcopy

# ============================================================ #
class PartialLockedDict:
    """
    THIS IS A BASECLASS
    
    PartialLockedDict conceptually consists one built-in dict and one outer dict.
    The built-in dict is considered constant and user are not suggested to change it.
    The outer dict is defined by the user and is sujected to all kinds of changes.
    PartialLockedDict provides methods accessing data with high security.
    
    
    
    METHODS
    ========
    these are all classmethods
    
    _error_change_builtin(cls, key):
        throws KeyError based on the input key.
        Call this function when built-in dict is going to get changed.
    
    _get_all(cls):
        * currently not implemented *
        return the dict covering both built-in dict and outer dict
        OUTPUT: * should be * dict (reference)
        
    _get_builtin(cls):
        * currently not implemented *
        return the built-in dict
        OUTPUT: * should be * dict (reference)
    
    reset(cls):
        * currently not implemented *
        clear the outer dict
        OUTPUT: * should be * cls
    
    getcopy(cls, key):
        retrieve the value with key in the whole dict
        INPUTS:  key, dict-key-like, the key to the value of interest
        OUTPUTS: value, dict-value-like, the value retrieved
    
    getcopy_all(cls):
        returns a deepcopy of the dict covering both built-in dict and outer dict
        OUTPUTS: dict (deepcopy)
        
    getcopy_builtin(cls):
        returns a deepcopy of the built-in dict
        OUTPUTS: dict (deepcopy)
        
    update(cls, key, value):
        updates the outer dict's dict-item indexed key with value
        if key in built-in dict, calls cls._error_change_builtin(cls, key) to raise KeyError
        INPUTS:  key, dict-key-like, the key to the item to be updated
                 value, dict-value-like, the value of update
        OUTPUTS: cls
    
    update_group(cls, dict_update):
        updates the outer dict with dict_update
        if dict_update contains any key that is also in the built-in dict, 
        calls cls._error_change_builtin(cls, key) to raise KeyError
        INPUTS:  dict_update, dict, the group of items as the update
        OUTPUTS: cls
        
    remove(cls, key):
        remove the key indexed dict-item in the outer dict
        if key in built-in dict, calls cls._error_change_builtin(cls, key) to raise KeyError
        INPUTS:  key, dict-key-like, the key to the item to be removed
        OUTPUTS: cls
    
    """
    @classmethod
    def _error_change_builtin(cls, key):
        raise KeyError
        
    @classmethod
    def _get_all(cls):
        raise NotImplementedError
        
    @classmethod
    def _get_builtin(cls):
        raise NotImplementedError
        
    @classmethod
    def reset(cls):
        raise NotImplementedError
        
    @classmethod
    def getcopy(cls, key):
        return cls._get_all()[key]
    
    @classmethod
    def getcopy_all(cls):
        return deepcopy(cls._get_all())
    
    @classmethod
    def getcopy_builtin(cls):
        return deepcopy(cls._get_builtin())
    
    @classmethod
    def update(cls, key, value):
        if key in cls._get_builtin():
            cls._error_change_builtin(key)
        cls._get_all()[key] = value
        return cls
    
    @classmethod
    def update_group(cls, dict_update):
        for key in dict_update.keys():
            if key in cls._get_builtin():
                cls._error_change_builtin(key)
        for key,value in dict_update.items():
            cls._get_all()[key] = value
        return cls
    
    @classmethod
    def remove(cls, key):
        if key in cls._get_builtin():
            cls._error_change_builtin(key)
        del cls._get_all()[key]
        return cls

In [ ]:
%%file Reaction.py
from BaseClass import PartialLockedDict
from copy import deepcopy
import numpy as np



class Reaction:
    """
    Reaction keeps all the infomation from one given reaction. It also helps to select 
    the right law function, as attribute rateCoeff, to compute the reaction rate coefficient 
    based on the classification of the given reaction. In the inner class _CoeffLaws several 
    laws including constant coeffs, Arrhenius coeffs and Modified Arrhenius coeffs have been 
    implemented, and _CoeffLaws forms up a dict-like structure to manage them.
    
    
    
    INPUTS
    =======
    
    ID:          str keyword, defaults 'reaction', reaction id
    
    reversible:  boolean keyword, defaults False, reversibility
                 if True, will raise NotImplementedError
                 
    TYPE:        str keyword, defaults 'Elementary', reaction type, 
                 if not 'Elementary', will raise NotImplementedError
                 
    coeffLaw:    str keyword, defaults 'Constant', 
                 name of the law that computes the reaction rate coefficients
                 if not in Reaction._CoeffLaws._dict_all, will raise NotImplementedError
                 
    coeffParams: dict keyword, defaults {}, param values required by coeffLaw
                 if not empty, should be in form of paramName(str): paramValue(float)
                 should not contain condition params such as temperature and concentration
    
    coeffUnits:  dict keyword, defaults {}, param units associated with coeffParams
                 if not empty, should be in form of paramName(str): paramUnit(str)
    
    reactants:   dict keyword, defaults {}, reactants
                 if not empty, should be in form of reactantName(str): stoichCoeff(int)
    
    products:    dict keyword, defaults {}, products
                 if not empty, should be in form of productName(str): stoichCoeff(int)
    
    kwargs:      some non-positional arguments that are currently not helpful
    
    
    
    ATTRIBUTES
    ===========
    
    rateCoeff: function, typically called as self.rateCoeff(**conditions)
        this function compute the reaction rate coefficient under certain condition.
        it is selected from the _CoeffLaws by __init__, and reseting self._params should 
        automatically reselect this rateCoeff. 
        --------------------------------------------
        For function rateCoeff(self, **conditions):
        INPUTS: conditions, non-positional params, usually contains:
            'T': float, temperature under which reaction happens
            'concs': array-like, chemical concentrations
        OUTPUTS: k, float, reaction rate coefficient
        
    _CoeffLaws: inner class, dict-like structure
        _CoeffLaws Keeps and manages the law functions that might be used to compute the 
        reaction rate coefficients. Each function is associated with a key - mostly their 
        name, in a string. It also provides several dict-like methods. All these will be 
        further specified inside the _CoeffLaws.
    
    _params: dict, records the parameters building up the instance.
        the keys of self._params are fixed to the following list and should not be changed:
        [KEYS] - reversible, TYPE, ID, coeffLaw, coeffParams, coeffUnits, reactants, products
    
    
    
    METHODS
    ========
    
    get_params(self): 
        return a deepcopy of self._params
        OUTPUTS: self._params, dict (deepcopy)
        
    set_params(self, **kwargs):
        update self._params with kwargs.
        only keys that are originally in _params would be updated.
        will call self._check_params() to see if this update is valid.
        if 'coeffLaw' is updated, will call self._specify_rateCoeff() to reset self.rateCoeff
        INPUTS:  kwargs, non-positional, contains the updates 
        OUTPUTS: self, Reaction instance
        
    getReactants(self): 
        returns the reactants in a dict
        OUTPUTS: self._params['reactants'], dict (deepcopy)
            has the form of (reactant name):(stoich coeff)
        
    getProducts(self): 
        returns the reactants in a dict
        OUTPUTS: self._params['products'], dict (deepcopy)
            has the form of (product name):(stoich coeff)
        
    _check_params(self):
        check if self._params are valid.
        it raises `NotImplementedError` if:
            self._params['reversible'] == True
            self._params['TYPE']       != 'Elementary'
            self._params['coeffLaw'] not in self._CoeffLaws._dict_all
            
    _specify_rateCoeff(self):
        select the right law to compute reaction rate coefficients, 
        and initialize it with self._params['coeffParams']. 
        self will get changed - attribute rateCoeff will be updated
        OUTPUTS: self.rateCoeff, function
            self.rateCoef(**conditions) will not need to take in 
            self._params['coeffParams'] as inputs
            
    __repr__(self):
        return a wrapped dict of all params, namely str(self._params)
        OUTPUTS: representational str, valid input for eval()
        
    __str__(self):
        return a str to show the contents of self. 
        when printed out, there will be two parts:
            the reaction equation, in a chemistry convention
            the params list, in a 'param name: param value' fashion
        OUTPUTS: descriptive str
    
    
    EXAMPLES
    =========
    >>> r = Reaction( \
            reactants=dict(H=1,O2=1), \
            products=dict(OH=1,H=1), \
            coeffLaw='Arrhenius', \
            coeffParams=dict(A=np.e, E=8.314)\
        )
    >>> r.rateCoeff(T=1.0)
    1.0
    >>> r.getReactants()
    {'H': 1, 'O2': 1}
    >>> r.set_params(reactants=dict(H=2,O2=2)).getReactants()
    {'H': 2, 'O2': 2}
    
    """
    
    class _CoeffLaws(PartialLockedDict):
        """
        _CoeffLaws keeps and manages the built-in methods that compute the reaction 
        rate coefficients. At the same time it also allow user to add their self-defined
        methods with similar usage. _CoeffLaws inherits the PartialLockedDict class.
        
        
        ATTRIBUTES
        ===========
        
        built-in functions includes:
            `Constant`     for constant coeffs
            `Arrhenius`    for Arrhenius coeffs
            `modArrhenius` for Modified Arrhenius coeffs
            
        _dict_builtin: dict
            the built-in dict which is not supposed to be changed.
            records mapping relation from names to the associated built-in functions,
            such as: ` 'Constant': const `
            
        _dict_all: dict
            a dict that contains what's in the built-in dict as well as user defined mappings.
            all searches and updates would be made on this dict.
            
            
        METHODS
        ========
        all methods are classmethods inherited from PartialLockedDict, among which the 
        following methods get further specification in this class:
        
        _error_change_builtin(cls, key):
            specified the message thrown by the KeyError
            input key will be clarified in that message
            
        _get_all(cls): 
            return _CoeffLaws._dict_all, as a reference
            OUTPUTS: _dict_all, dict (reference)
            
        _get_builtin(cls):
            return _CoeffLaws._dict_builtin, as a reference
            OUTPUTS: _dict_builtin, dict (reference)
            
            
        EXAMPLES
        =========
        >>> def somelaw(T, A=1.0): return T * A
        >>> Reaction._CoeffLaws.update('sl', somelaw).getcopy_all().keys()
        dict_keys(['Constant', 'Arrhenius', 'modArrhenius', 'sl'])
        >>> r = Reaction( \
            reactants=dict(H=1,O2=1), \
            products=dict(OH=1,H=1), \
            coeffLaw='sl', \
            coeffParams=dict(A=2.0)\
        )
        >>> r.rateCoeff(T=0.1)
        0.2
        >>> Reaction._CoeffLaws.reset().getcopy_all().keys()
        dict_keys(['Constant', 'Arrhenius', 'modArrhenius'])
        """
        

        def const(k=1.0):
            if k <= 0.0:
                raise ValueError(' '.join([
                    'k = {0:18.16e}:'.format(k), 
                    'Non-positive reaction rate coefficient is prohibited.']))
            return k
        def arr(T, R=8.314, A=1.0, E=0.0):
            if T <= 0.0:
                raise ValueError(' '.join([
                    'T = {0:18.16e}:'.format(T),
                    'Non-positive temperature is prohibited.']))
            if A <= 0.0:
                raise ValueError(' '.join([
                    'A = {0:18.16e}:'.format(A),
                    'Non-positive Arrhenius prefactor is prohibited.']))
            if R <= 0.0:
                raise ValueError(' '.join([
                    'R = {0:18.16e}:'.format(R),
                    'Non-positive ideal gas constant is prohibited.']))
            return A * np.exp(-E / (R * T))
        def modarr(T, R=8.314, A=1.0, b=0.0, E=0.0):
            if T <= 0.0:
                raise ValueError(' '.join([
                    'T = {0:18.16e}:'.format(T),
                    'Non-positive temperature is prohibited.']))
            if A <= 0.0:
                raise ValueError(' '.join([
                    'A = {0:18.16e}:'.format(A),
                    'Non-positive Arrhenius prefactor is prohibited.']))
            if R <= 0.0:
                raise ValueError(' '.join([
                    'R = {0:18.16e}:'.format(R),
                    'Non-positive ideal gas constant is prohibited.']))
            return A * (T ** b) * np.exp(-E / (R * T))
            
        _dict_builtin = {
            'Constant'    :const, 
            'Arrhenius'   :arr, 
            'modArrhenius':modarr
        }
        _dict_all = deepcopy(_dict_builtin) 
        
        @classmethod
        def _error_change_builtin(cls, key):
            raise KeyError(' '.join([
                'LawName = {}'.format(key),
                'exists as a built-in law.',
                'Changing a built-in law is prohibited.']))
        @classmethod
        def _get_all(cls):
            return cls._dict_all
        @classmethod
        def _get_builtin(cls):
            return cls._dict_builtin
        @classmethod
        def reset(cls):
            cls._dict_all = deepcopy(cls._dict_builtin)
            return cls
            
    
    def __init__(
        self, 
        reversible  = False, 
        TYPE        = 'Elementary', 
        ID          = 'reaction', 
        coeffLaw    = 'Constant', 
        coeffParams = {},
        coeffUnits  = {},
        reactants   = {},
        products    = {},
        **kwargs
    ):
        self._params = {
            'reversible'  : reversible, 
            'TYPE'        : TYPE, 
            'ID'          : ID, 
            'coeffLaw'    : coeffLaw, 
            'coeffParams' : coeffParams,
            'coeffUnits'  : coeffUnits,
            'reactants'   : reactants,
            'products'    : products
        }
        self._check_params()
        self._specify_rateCoeff()
        # this brings up the attribute self.rateCoeff
    
    def get_params(self):
        return deepcopy(self._params)
    def set_params(self, **kwargs):
        for k in kwargs.keys():
            if k not in self._params:
                kwargs.pop(k)
        self._params.update(**kwargs)
        self._check_params()
        self._specify_rateCoeff()
        return self
    
    def getReactants(self):
        return self._params['reactants']
    def getProducts(self):
        return self._params['products']
    
    def _check_params(self):
        if self._params['reversible']:
            raise NotImplementedError(
                'Reversible reaction is not implemented.')
        if self._params['TYPE'] != 'Elementary':
            raise NotImplementedError(' '.join([
                'TYPE = {}.'.format(self._params['TYPE']),
                'Non-elementary reaction is not implemented.']))
        if not self._params['coeffLaw'] in self._CoeffLaws._dict_all:
            raise NotImplementedError(' '.join([
                'coeffLaw = {}.'.format(self._params['coeffLaw']),
                'Refered reaction rate coefficient law is not implemented.']))
        return deepcopy(self._params)
    def _specify_rateCoeff(self):
        def rateCoeff_specified(**conditions):
            selection = self._params['coeffLaw']
            params = self._params['coeffParams']
            return self._CoeffLaws._dict_all[selection](**params, **conditions)
        self.rateCoeff = rateCoeff_specified
        return self.rateCoeff
    
    def __repr__(self):
        return str(self._params)
    
    def __str__(self):
        streq_lefthand = ' + '.join(
            ['{}{}'.format(v,k) for k,v in self._params['reactants'].items()])
        streq_righthand = ' + '.join(
            ['{}{}'.format(v,k) for k,v in self._params['products'].items()])
        streq_full = ' [=] '.join([streq_lefthand, streq_righthand])
        strparams = '\n'.join(
            [': '.join([str(k), str(v)]) for k,v in self._params.items()])
        return '\n'.join([
            '=' * 40, 'Reaction Equation:', streq_full, 
            '-' * 40, 'Reaction Info:', strparams,
            '=' * 40])

In [ ]:
!pytest --doctest-modules

In [ ]:
from Reaction import Reaction
import numpy as np
r = Reaction(
    reactants=dict(H=1,O2=1), products=dict(OH=1,H=1),
    coeffLaw='Arrhenius', coeffParams=dict(E=-8.314)
)
print(str(r))
print(r.rateCoeff(T=1.0))
print(r.getReactants())

In [ ]:
# novel coeffLaw input would raise NotImplementedError
try:
    Reaction._CoeffLaws.reset()
    _r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
except NotImplementedError as err:
    print(err)

In [ ]:
# user can define his own coeffLaw and use _CoeffLaws.update to register it in the Reaction class
# this won't raise NotImplementedError
# also user may input temperature as numpy.array
def _law(**kwargs):
    return 3.14 * kwargs['A'] * kwargs['T']

Reaction._CoeffLaws.update('_', _law)
_r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
_r.rateCoeff(T=np.array([2,3]))

In [ ]:
%%file test_Reaction.py
from Reaction import Reaction
import numpy as np

def ListTest(func):
    # func() is a test function that returns list of test results
    def inner():
        for t in func():
            assert(t)
    return inner

# ============ Tests on Results ============ #

@ListTest
def test_info():
    r = Reaction(reactants=dict(H=1,O2=1), products=dict(OH=1,H=1))
    return [
        r.getReactants() == dict(H=1,O2=1),
        r.getProducts() == dict(OH=1,H=1)]

@ListTest
def test_rateCeff():
    r1 = Reaction(coeffLaw='Constant', coeffParams=dict(k=3.14))
    r2 = Reaction(coeffLaw='Arrhenius', coeffParams=dict(E=8.314))
    r3 = Reaction(coeffLaw='modArrhenius', coeffParams=dict(b=3,E=2*8.314))
    return [
        r1.rateCoeff() == 3.14,
        r2.rateCoeff(T=1.0) == 1/np.e,
        r3.rateCoeff(T=2.0) == 8/np.e]

@ListTest
def test_CoeffLaws_get():
    return [
        Reaction._CoeffLaws.getcopy('Arrhenius') == Reaction._CoeffLaws.arr,
        Reaction._CoeffLaws.getcopy_all() == Reaction._CoeffLaws._dict_all,
        Reaction._CoeffLaws.getcopy_builtin() == Reaction._CoeffLaws._dict_builtin]

@ListTest
def test_CoeffLaws_update_remove_reset():
    def _law1(**kwargs): return 0.0
    def _law2(**kwargs): return 0.0
    Reaction._CoeffLaws.reset()
    Reaction._CoeffLaws.update('_1',_law1)
    Reaction._CoeffLaws.update_group(dict(_1=_law1,_2=_law2))
    test = [
        '_1' in Reaction._CoeffLaws._dict_all,
        '_2' in Reaction._CoeffLaws._dict_all,
        '_1' not in Reaction._CoeffLaws._dict_builtin] 
    Reaction._CoeffLaws.remove('_1')
    test += [
        '_1' not in Reaction._CoeffLaws._dict_all,
        '_2' in Reaction._CoeffLaws._dict_all]
    Reaction._CoeffLaws.reset()
    test += [
        Reaction._CoeffLaws._dict_all == Reaction._CoeffLaws._dict_builtin]
    return test


# ============ Tests on Errors ============ #

def test_init_notimplemented():
    try:
        Reaction(reversible=True)
    except NotImplementedError as err:
        assert(type(err) == NotImplementedError)
    try:
        Reaction(**{'TYPE':'duplicate'})
    except NotImplementedError as err:
        assert(type(err) == NotImplementedError)
    try:
        Reaction(coeffLaw='_')
    except NotImplementedError as err:
        assert(type(err) == NotImplementedError)
        
def test_CoeffLaws_changebuiltin():
    def _law1(**kwargs): return 0.0
    def _law2(**kwargs): return 0.0
    Reaction._CoeffLaws.reset()
    try:
        Reaction._CoeffLaws.update('Arrhenius',_law1)
    except KeyError as err:
        assert(type(err) == KeyError)
    try:
        Reaction._CoeffLaws.update_group(dict(_1=_law1,arr=_law2))
    except KeyError as err:
        assert(type(err) == KeyError)
    try:
        Reaction._CoeffLaws.remove('Arrhenius')
    except KeyError as err:
        assert(type(err) == KeyError)
    Reaction._CoeffLaws.reset()
        
def test_CoeffLaws_input():
    try:
        Reaction(coeffLaw='Constant', coeffParams=dict(k=-1.0)).rateCoeff()
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='Arrhenius').rateCoeff(T=-1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='Arrhenius', coeffParams=dict(A=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='Arrhenius', coeffParams=dict(R=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='modArrhenius').rateCoeff(T=-1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='modArrhenius', coeffParams=dict(A=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)
    try:
        Reaction(coeffLaw='modArrhenius', coeffParams=dict(R=-1.0)).rateCoeff(T=1.0)
    except ValueError as err:
        assert (type(err) == ValueError)

In [ ]:
!pytest --cov --cov-report term-missing

In [56]:
%%file Check.py
class ValueCheck:
    def __init__(self, criteron, name):
        # criteron should be a function type
        # criteron(x) returns boolean value
        self._criteron = criteron
        self._name = name
    def response(self, x, label, term):
        if not self._criteron(x):
            raise ValueError(
                '{} = {:18.16e}: {} {} is prohibited.' \
                .format(label, x, self._name, term))

Writing Check.py


In [24]:
%%file CoeffLaw.py
from Check import ValueCheck
import numpy as np

class CoeffLaw:
    def __init__(self, check=True, **coeffparams):
        if check:
            self.check_coeffparams(**coeffparams)
        self._coeffparams = coeffparams
    def compute(self, check=True, **stateparams):
        if check: 
            self.check_stateparams(**stateparams)
        return self._kernel(**self._coeffparams, **stateparams)
    @staticmethod
    def check_coeffparams(**coeffparams):
        pass
    @staticmethod
    def check_stateparams(**stateparams):
        pass
    @staticmethod
    def _kernel(**params):
        raise NotImplementedError


class Arrhenius(CoeffLaw):
    
    _check_np = ValueCheck(lambda x:x>=0.0, 'non-positive')
    
    def __init__(self, check=True, A=1.0, E=0.0, R=8.314, **other_params):
        if check:
            self.check_coeffparams(A, R)
        self._A, self._E, self._R = A, E, R
    def compute(self, check=True, T=1e-16, **other_params):
        if check: 
            self.check_stateparams(T)
        return self._kernel(T, self._A, self._E, self._R)
    @staticmethod
    def check_coeffparams(A, R, **other_params):
        Arrhenius._check_np.response(R, 'R', 'ideal gas constant')
        Arrhenius._check_np.response(A, 'A', 'Arrhenius prefactor')
    @staticmethod
    def check_stateparams(T, **other_params):
        Arrhenius._check_np.response(T, 'T', 'temperature')
    @staticmethod
    def _kernel(T, A, E, R, **other_params):
        return A * (np.e ** (-E / (R * T)))

    
class modArrhenius(CoeffLaw):
    
    _check_np = ValueCheck(lambda x:x>=0.0, 'non-positive')
    
    def __init__(self, check=True, A=1.0, b=0.0, E=0.0, R=8.314, **other_params):
        if check:
            self.check_coeffparams(A, R)
        self._A, self._b, self._E, self._R = A, b, E, R
    def compute(self, check=True, T=1e-16, **other_params):
        if check: 
            self.check_stateparams(T)
        return self._kernel(T, self._A, self._b, self._E, self._R)
    @staticmethod
    def check_coeffparams(A, R, **other_params):
        modArrhenius._check_np.response(A, 'A', 'Arrhenius prefactor')
        modArrhenius._check_np.response(R, 'R', 'ideal gas constant')
    @staticmethod
    def check_stateparams(T, **other_params):
        modArrhenius._check_np.response(T, 'T', 'temperature')
    @staticmethod
    def _kernel(T, A, b, E, R, **other_params):
        return A * (T ** b) * (np.e ** (-E / (R * T)))
    
    
class Constant(CoeffLaw):
    
    _check_np = ValueCheck(lambda x:x>=0.0, 'non-positive')
    
    def __init__(self, check=True, k=1.0, **other_params):
        if check:
            self.check_coeffparams(k)
        self._k = k
    def compute(self, **other_params):
        return self._k
    @staticmethod
    def check_coeffparams(k, **other_params):
        Constant._check_np.response(k, 'k', 'reaction rate coefficient')
    @staticmethod
    def _kernel(k, **other_params):
        return k

Overwriting CoeffLaw.py


In [1]:
import CoeffLaw
arr = CoeffLaw.Constant(k=-1.0, **dict(A=-1.0, b=1.0, E=-8.314, p=0.05, t=1.96), check=0)
print('{:18.16e}'.format(arr.compute(T=1.0, check=True)))
# arr._A

-1.0000000000000000e+00


In [14]:
%%file Reaction_LawStructured.py
from BaseClass import PartialLockedDict
from copy import deepcopy
import numpy as np
import CoeffLaw


class Reaction:
    """
    Reaction keeps all the infomation from one given reaction. It also helps to select 
    the right law function, as attribute rateCoeff, to compute the reaction rate coefficient 
    based on the classification of the given reaction. In the inner class _CoeffLawDict several 
    laws including constant coeffs, Arrhenius coeffs and Modified Arrhenius coeffs have been 
    implemented, and _CoeffLawDict forms up a dict-like structure to manage them.
    
    
    
    INPUTS
    =======
    
    ID:          str keyword, defaults 'reaction', reaction id
    
    reversible:  boolean keyword, defaults False, reversibility
                 if True, will raise NotImplementedError
                 
    TYPE:        str keyword, defaults 'Elementary', reaction type, 
                 if not 'Elementary', will raise NotImplementedError
                 
    coeffLaw:    str keyword, defaults 'Constant', 
                 name of the law that computes the reaction rate coefficients
                 if not in Reaction._CoeffLawDict._dict_all, will raise NotImplementedError
                 
    coeffParams: dict keyword, defaults {}, param values required by coeffLaw
                 if not empty, should be in form of paramName(str): paramValue(float)
                 should not contain state params such as temperature and concentration
                 coeffParams will be checked based on the input of coeffLaw,
                 if such check cannot pass, ValueError will be raised
    
    coeffUnits:  dict keyword, defaults {}, param units associated with coeffParams
                 if not empty, should be in form of paramName(str): paramUnit(str)
    
    reactants:   dict keyword, defaults {}, reactants
                 if not empty, should be in form of reactantName(str): stoichCoeff(int)
    
    products:    dict keyword, defaults {}, products
                 if not empty, should be in form of productName(str): stoichCoeff(int)
    
    kwargs:      some non-positional arguments that are currently not helpful
    
    
    
    ATTRIBUTES
    ===========
    
    rateCoeff: function, typically called as self.rateCoeff(**state)
        this function compute the reaction rate coefficient under certain state.
        it is selected from the _CoeffLawDict by __init__, and reseting self._params should 
        automatically reselect this rateCoeff. 
        --------------------------------------------
        For function rateCoeff(self, **state):
        INPUTS: state, non-positional params, usually contains:
            'T': float, temperature under which reaction happens
            'concs': array-like, chemical concentrations
        OUTPUTS: k, float, reaction rate coefficient
        
    _CoeffLawDict: inner class, dict-like structure
        _CoeffLawDict Keeps and manages the law functions that might be used to compute the 
        reaction rate coefficients. Each function is associated with a key - mostly their 
        name, in a string. It also provides several dict-like methods. All these will be 
        further specified inside the _CoeffLawDict.
    
    _params: dict, records the parameters building up the instance.
        the keys of self._params are fixed to the following list and should not be changed:
        [KEYS] - reversible, TYPE, ID, coeffLaw, coeffParams, coeffUnits, reactants, products
    
    
    
    METHODS
    ========
    
    get_params(self): 
        return a deepcopy of self._params
        OUTPUTS: self._params, dict (deepcopy)
        
    set_params(self, **kwargs):
        update self._params with kwargs.
        only keys that are originally in _params would be updated.
        will call self._check_params() to see if this update is valid.
        if 'coeffLaw' is updated, will call self._specify_rateCoeff() to reset self.rateCoeff
        INPUTS:  kwargs, non-positional, contains the updates 
        OUTPUTS: self, Reaction instance
        
    getReactants(self): 
        returns the reactants in a dict
        OUTPUTS: self._params['reactants'], dict (deepcopy)
            has the form of (reactant name):(stoich coeff)
        
    getProducts(self): 
        returns the reactants in a dict
        OUTPUTS: self._params['products'], dict (deepcopy)
            has the form of (product name):(stoich coeff)
        
    _check_params(self):
        check if self._params are valid.
        it raises `NotImplementedError` if:
            self._params['reversible'] == True
            self._params['TYPE']       != 'Elementary'
            self._params['coeffLaw'] not in self._CoeffLawDict._dict_all
            
    _specify_rateCoeff(self):
        select the right law to compute reaction rate coefficients, 
        and initialize it with self._params['coeffParams']. 
        self will get changed - attribute rateCoeff will be updated
        OUTPUTS: self.rateCoeff, function
            self.rateCoef(**state) will not need to take in 
            self._params['coeffParams'] as inputs
            
    __repr__(self):
        return a wrapped dict of all params, namely str(self._params)
        OUTPUTS: representational str, valid input for eval()
        
    __str__(self):
        return a str to show the contents of self. 
        when printed out, there will be two parts:
            the reaction equation, in a chemistry convention
            the params list, in a 'param name: param value' fashion
        OUTPUTS: descriptive str
    
    
    EXAMPLES
    =========
    >>> r = Reaction( \
            reactants=dict(H=1,O2=1), \
            products=dict(OH=1,H=1), \
            coeffLaw='Arrhenius', \
            coeffParams=dict(A=np.e, E=8.314)\
        )
    >>> r.rateCoeff(T=1.0)
    1.0
    >>> r.getReactants()
    {'H': 1, 'O2': 1}
    >>> r.set_params(reactants=dict(H=2,O2=2)).getReactants()
    {'H': 2, 'O2': 2}
    
    """
    
    class _CoeffLawDict(PartialLockedDict):
        """
        _CoeffLawDict keeps and manages the built-in methods that compute the reaction 
        rate coefficients. At the same time it also allow user to add their self-defined
        methods with similar usage. _CoeffLawDict inherits the PartialLockedDict class.
        
        
        ATTRIBUTES
        ===========
        
        built-in functions includes:
            `Constant`     for constant coeffs
            `Arrhenius`    for Arrhenius coeffs
            `modArrhenius` for Modified Arrhenius coeffs
            
        _dict_builtin: dict
            the built-in dict which is not supposed to be changed.
            records mapping relation from names to the associated built-in functions,
            such as: ` 'Constant': const `
            
        _dict_all: dict
            a dict that contains what's in the built-in dict as well as user defined mappings.
            all searches and updates would be made on this dict.
            
            
        METHODS
        ========
        all methods are classmethods inherited from PartialLockedDict, among which the 
        following methods get further specification in this class:
        
        _error_change_builtin(cls, key):
            specified the message thrown by the KeyError
            input key will be clarified in that message
            
        _get_all(cls): 
            return _CoeffLawDict._dict_all, as a reference
            OUTPUTS: _dict_all, dict (reference)
            
        _get_builtin(cls):
            return _CoeffLawDict._dict_builtin, as a reference
            OUTPUTS: _dict_builtin, dict (reference)
            
            
        EXAMPLES
        =========
        >>> class somelaw(CoeffLaw.CoeffLaw): \
                @staticmethod \
                def _kernel(T, A, **other_params): \
                    return T * A \
        >>> Reaction._CoeffLawDict.update('sl', somelaw).getcopy_all().keys()
        dict_keys(['Constant', 'Arrhenius', 'modArrhenius', 'sl'])
        >>> r = Reaction( \
            reactants=dict(H=1,O2=1), \
            products=dict(OH=1,H=1), \
            coeffLaw='sl', \
            coeffParams=dict(A=2.0)\
        )
        >>> r.rateCoeff(T=0.1)
        0.2
        >>> Reaction._CoeffLawDict.reset().getcopy_all().keys()
        dict_keys(['Constant', 'Arrhenius', 'modArrhenius'])
        """
            
        _dict_builtin = {
            'Constant'    :CoeffLaw.Constant, 
            'Arrhenius'   :CoeffLaw.Arrhenius, 
            'modArrhenius':CoeffLaw.modArrhenius
        }
        _dict_all = deepcopy(_dict_builtin) 
        
        @classmethod
        def _error_change_builtin(cls, key):
            raise KeyError(' '.join([
                'LawName = {}'.format(key),
                'exists as a built-in law.',
                'Changing a built-in law is prohibited.']))
        @classmethod
        def _get_all(cls):
            return cls._dict_all
        @classmethod
        def _get_builtin(cls):
            return cls._dict_builtin
        @classmethod
        def reset(cls):
            cls._dict_all = deepcopy(cls._dict_builtin)
            return cls
            
    
    def __init__(
        self, 
        reversible  = False, 
        TYPE        = 'Elementary', 
        ID          = 'reaction', 
        coeffLaw    = 'Constant', 
        coeffParams = {},
        coeffUnits  = {},
        reactants   = {},
        products    = {},
        **kwargs
    ):
        self._params = {
            'reversible'  : reversible, 
            'TYPE'        : TYPE, 
            'ID'          : ID, 
            'coeffLaw'    : coeffLaw, 
            'coeffParams' : coeffParams,
            'coeffUnits'  : coeffUnits,
            'reactants'   : reactants,
            'products'    : products
        }
        self._check_params()
        self._specify_rateCoeff()
        # this brings up the attribute self.rateCoeff
    
    def get_params(self):
        return deepcopy(self._params)
    def set_params(self, **kwargs):
        for k in kwargs.keys():
            if k not in self._params:
                kwargs.pop(k)
        self._params.update(**kwargs)
        self._check_params()
        self._specify_rateCoeff()
        return self
    
    def getReactants(self):
        return self._params['reactants']
    def getProducts(self):
        return self._params['products']
    
    def _check_params(self):
        if self._params['reversible']:
            raise NotImplementedError(
                'Reversible reaction is not implemented.')
        if self._params['TYPE'] != 'Elementary':
            raise NotImplementedError(' '.join([
                'TYPE = {}.'.format(self._params['TYPE']),
                'Non-elementary reaction is not implemented.']))
        if not self._params['coeffLaw'] in self._CoeffLawDict._dict_all:
            raise NotImplementedError(' '.join([
                'coeffLaw = {}.'.format(self._params['coeffLaw']),
                'Refered reaction rate coefficient law is not implemented.']))
        return deepcopy(self._params)
    def _specify_rateCoeff(self):
        def rateCoeff_specified(check=True, **stateparams):
            coeffparams = self._params['coeffParams']
            selection = self._CoeffLawDict._dict_all[self._params['coeffLaw']]
            return selection(**coeffparams).compute(check, **stateparams)
        self.rateCoeff = rateCoeff_specified
        return self.rateCoeff
    
    def __repr__(self):
        return str(self._params)
    
    def __str__(self):
        streq_lefthand = ' + '.join(
            ['{}{}'.format(v,k) for k,v in self._params['reactants'].items()])
        streq_righthand = ' + '.join(
            ['{}{}'.format(v,k) for k,v in self._params['products'].items()])
        streq_full = ' [=] '.join([streq_lefthand, streq_righthand])
        strparams = '\n'.join(
            [': '.join([str(k), str(v)]) for k,v in self._params.items()])
        return '\n'.join([
            '=' * 40, 'Reaction Equation:', streq_full, 
            '-' * 40, 'Reaction Info:', strparams,
            '=' * 40])

Overwriting Reaction_LawStructured.py


In [15]:
!pytest --doctest-modules

============================= test session starts ==============================
platform darwin -- Python 3.6.1, pytest-3.0.7, py-1.4.33, pluggy-0.4.0
rootdir: /Users/shell/Documents/GitHub/cs207-FinalProject, inifile: setup.cfg
plugins: cov-2.5.1
collected 11 items 

Reaction.py ..
Reaction_LawStructured.py .F
test_Reaction.py .......Coverage.py warning: Module roots was never imported. (module-not-imported)
Coverage.py warning: No data was collected. (no-data-collected)


=================================== FAILURES ===================================
___________ [doctest] Reaction_LawStructured.Reaction._CoeffLawDict ____________
182             OUTPUTS: _dict_all, dict (reference)
183             
184         _get_builtin(cls):
185             return _CoeffLawDict._dict_builtin, as a reference
186             OUTPUTS: _dict_builtin, dict (reference)
187             
188             
189         EXAMPLES
190         =========
191         >>> class somelaw(CoeffLaw.CoeffLaw):       

In [3]:
# from Reaction_LawStructured import Reaction
import numpy as np
r = Reaction(
    reactants=dict(H=1,O2=1), products=dict(OH=1,H=1),
    coeffLaw='modArrhenius', coeffParams=dict(b=1.0, E=8.314)
)
print(str(r))
print(r.rateCoeff(check=0, T=-1.0))
print(r.getReactants())

Reaction Equation:
1H + 1O2 [=] 1OH + 1H
----------------------------------------
Reaction Info:
reversible: False
TYPE: Elementary
ID: reaction
coeffLaw: modArrhenius
coeffParams: {'b': 1.0, 'E': 8.314}
coeffUnits: {}
reactants: {'H': 1, 'O2': 1}
products: {'OH': 1, 'H': 1}
-2.718281828459045
{'H': 1, 'O2': 1}


In [4]:
try:
    Reaction._CoeffLawDict.reset()
    _r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
except NotImplementedError as err:
    print(err)

coeffLaw = _. Refered reaction rate coefficient law is not implemented.


In [9]:
import CoeffLaw
class _law(CoeffLaw.CoeffLaw):
    @staticmethod
    def _kernel(T, A, **other_params):
        return T * A

Reaction._CoeffLawDict.update('_', _law)
_r = Reaction(coeffLaw='_', coeffParams=dict(A=0.1))
_r.rateCoeff(T=2.0)
# _r._CoeffLawDict._dict_all[_r._params['coeffLaw']](_r._params['coeffParams']).compute(T=2,A=0.1)

0.2

In [100]:
class f:
    @staticmethod
    def dosth():
        return 3.14
    @staticmethod
    def dupdosth():
        return f.dosth()
    def tridoth(self):
        return self.dupdosth()

fin = f()
fin.tridoth()

3.14

In [ ]:
a = lambda x:x>0

In [ ]:
a(3)

In [10]:
def func(a, b):
    return a+b
func(**dict(b=1,a=3,c=4))

TypeError: func() got an unexpected keyword argument 'c'

In [9]:
None == 0

False

In [12]:
a=3
dict(a=a)

{'a': 3}

In [18]:
import numpy as np
np.exp(-1/1e-7)

0.0

In [69]:
def fo(**kws, a=0):
    x=0
    def fi():
        return x + 1
    return fi
fo()()

SyntaxError: invalid syntax (<ipython-input-69-b0eb66b5ea15>, line 1)